In [321]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from selenium.webdriver.support.wait import WebDriverWait
from bs4 import BeautifulSoup
import pandas as pd
import time 

In [3]:
driver=webdriver.Chrome()

In [18]:
driver.get("https://www.yelp.com/")

In [332]:
def yelp_scrap(service, location, pages):
    restaurant_name=[]
    phone=[]
    address=[]
    ratings=[]
    links=[]
    pg=10
    driver.get("https://www.yelp.com/")
    driver.find_element(By.NAME, "find_desc").send_keys(service+Keys.ENTER)
    driver.find_element(By.ID, "search_location").send_keys(Keys.CONTROL, "a")
    driver.find_element(By.ID, "search_location").send_keys(Keys.DELETE)
    driver.find_element(By.ID, "search_location").send_keys(location+Keys.ENTER)
    time.sleep(2)
    soup=BeautifulSoup(driver.page_source, "html.parser")
    a=soup.find("div", {"role": "navigation"})
    b=int(a.text.split(" ")[-1])
    if pages>b:
        print(f"There are only {b} pages present on the webpage. Scrapping {b} pages!")
        pages=b
    site=driver.current_url
    for z in range(0,pages*10,10):
        c=[]
        elements=[]
        baseurl="https://www.yelp.com"
        driver.get(f"{site}&start={z}")
        time.sleep(1)
        soup=BeautifulSoup(driver.page_source, "html.parser")
        for i in soup.find_all("a",{"class":"css-19v1rkv"}):
            c.append(i)
        c=c[2:]
        for i in c:
            if len(str(i))<270:
                elements.append(str(i))
        for i in elements:
            i=i.split(" ")
            d=i[2]
            if d[0]=="h":
                links.append(f"{baseurl}{d[6:-1]}")
    for x in links:
        driver.get(x)
        soup=BeautifulSoup(driver.page_source, "html.parser")
        restaurant_name.append(soup.find("h1", {"class": "css-1se8maq"}).text)
        phone.append(driver.find_elements(By.XPATH, '//p[@class=" css-1p9ibgf"][1]')[1].text)
        address.append(driver.find_element(By.XPATH, '//p[@class=" css-qyp8bo"]').text)
    df=pd.DataFrame({"Restaurant_Name": restaurant_name, "Phone_Number": phone, "Address": address})
    df.to_excel("Yelp Scrap Data.xlsx", sheet_name="1")

In [ ]:
service=input(f"Kindly enter the name of the service required: ")
locality=input(f"Kindly enter the name of the locality: ")
pag=int(input("Kindly enter the number of pages to be scrapped: "))
yelp_scrap(service, locality, pag)